In [14]:
%reload_ext autoreload
%autoreload 2

from func import *
import pandas as pd
import pickle

In [15]:
filename = 'all_data_anonymized.csv'
date_col = 'todate'
start_date = '1980-01-01'
seperator = ';'
fill = 0
lim = 5
df_read = load_and_preprocess_data(filename, date_col, start_date, seperator, fill, lim)


d:\master3\Master-thesis\Files\func.py:21: DtypeWarning: Columns (15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_read = pd.read_csv(filename, sep=seperator)


In [16]:
windows = [5, 10, 20, 40, 60, 100, 180, 240, 360, 480]
window_m = [10, 30, 60, 100, 180]
assets = df_read.columns
df_feat = add_features(df_read, window_m)

In [17]:
target_days = 9

return_column_shift = 'avgreturn'
volatility_column_shift = 'volatility'
df = add_y_col(df_feat, df_read, date_col, target_days, return_column_shift, volatility_column_shift)

In [18]:
X_train, X_eval, X_test, y_train, y_eval, y_test, X_train_eval, y_train_eval = prepare_training_dataset(df, date_col, shuffle=False, train_split=0.25, eval_split=0.25)

In [19]:
param_grid_alpha = {'alpha': [0.1, 1.0, 10.0]}
ridge_best, grid_search = optimize_and_train_ridge(X_train, y_train, X_train_eval, y_train_eval, param_grid_alpha)

# save the model to pickle


filename = 'ridge_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(ridge_best, file)


Best parameters: {'alpha': 10.0}
Best accuracy on evaluation set: 0.5086063469241039


In [20]:
param_grid_xgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_best, best_params = optimize_and_train_xgb(X_train, y_train, X_eval, y_eval, param_grid_xgb)

# save the model to pickle

filename = 'xgb_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(xgb_best, file)

C:\Users\Oscar Azrak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [21]:

param_grid_lgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 63, 127, 255]
}

# Call the function with your datasets and hyperparameter grid
lgb_best, best_params = optimize_and_train_lgb(X_train, y_train, X_eval, y_eval, param_grid_lgb)

# save the model to pickle

filename = 'lgb_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(lgb_best, file)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 47869, number of negative: 49501
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007959 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8670
[LightGBM] [Info] Number of data points in the train set: 97370, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.491620 -> initscore=-0.033525
[LightGBM] [Info] Start training from score -0.033525
Best hyperparameters: {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 200, 'num_leaves': 31}
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Wa

In [13]:
NN_model, history, X_test_scaled = train_and_evaluate_NN(X_train_eval, y_train_eval, X_eval, y_eval, X_test, y_test)

# save the model 

model_filename = 'NN_model.h5'
NN_model.save(model_filename)

X_test_scaled_filename = 'X_test_scaled.pickle'
with open(X_test_scaled_filename, 'wb') as file:
    pickle.dump(X_test_scaled, file)

Epoch 1/50


C:\Users\Oscar Azrak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6086/6086 ━━━━━━━━━━━━━━━━━━━━ 5s 715us/step - accuracy: 0.5239 - loss: 0.2493 - val_accuracy: 0.5396 - val_loss: 0.2478
Epoch 2/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 670us/step - accuracy: 0.5321 - loss: 0.2484 - val_accuracy: 0.5456 - val_loss: 0.2470
Epoch 3/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 676us/step - accuracy: 0.5416 - loss: 0.2475 - val_accuracy: 0.5390 - val_loss: 0.2476
Epoch 4/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 662us/step - accuracy: 0.5449 - loss: 0.2468 - val_accuracy: 0.5566 - val_loss: 0.2454
Epoch 5/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 673us/step - accuracy: 0.5523 - loss: 0.2460 - val_accuracy: 0.5566 - val_loss: 0.2451
Epoch 6/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 668us/step - accuracy: 0.5533 - loss: 0.2456 - val_accuracy: 0.5604 - val_loss: 0.2443
Epoch 7/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 672us/step - accuracy: 0.5612 - loss: 0.2442 - val_accuracy: 0.5673 - val_loss: 0.2435
Epoch 8/50
6086/6086 ━━━━━━━━━━━━━━━━━━━━ 4s 676us/step - accuracy: 0.5622 - loss: 0.24